In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join("..", "..")))
from nlp.preprocessing import hg_encoder
import json

In [3]:
r = open("../../datasets/extractive_summary/train.jsonl", "r", encoding="utf8")
w = open("../../datasets/extractive_summary/train_hg.json", "a", encoding="utf8")
c = open("../../datasets/extractive_summary/collection.txt", "a", encoding="utf8")
try:
    count = 0
    # while True:
    line = r.readline()
    # if not line: break
    count += 1
    line = json.loads(line)
    for i, s in enumerate(line["article_original"]):
        line["article_original"][i] = hg_encoder.encode(s)
        c.write(line["article_original"][i] + "\n")
    line["abstractive"] = hg_encoder.encode(line["abstractive"])
    c.write(line["abstractive"] + "\n")
    w.write(json.dumps(line, ensure_ascii=False))
    if count % 100 == 0:
        print(f"█{count}")
    else:
        print("█", end="")
    print(f"data size: {count}")
except Exception as e:
    raise e
finally:
    r.close()
    w.close()
    c.close()

█data size: 1


In [10]:
line

''